In [30]:
import os
from os import path as osp
import numpy as np

from mmengine.config import Config, DictAction
from mmengine.registry import init_default_scope
from mmengine.utils import ProgressBar, mkdir_or_exist

from mmdet3d.registry import DATASETS, VISUALIZERS
from mmdet3d.utils import replace_ceph_backend

In [3]:
os.chdir('../')

In [4]:
def build_data_cfg(config_path, aug, cfg_options):
    """Build data config for loading visualization data."""

    cfg = Config.fromfile(config_path)
    if cfg_options is not None:
        cfg.merge_from_dict(cfg_options)

    # extract inner dataset of `RepeatDataset` as
    # `cfg.train_dataloader.dataset` so we don't
    # need to worry about it later
    if cfg.train_dataloader.dataset['type'] == 'RepeatDataset':
        cfg.train_dataloader.dataset = cfg.train_dataloader.dataset.dataset
    # use only first dataset for `ConcatDataset`
    if cfg.train_dataloader.dataset['type'] == 'ConcatDataset':
        cfg.train_dataloader.dataset = cfg.train_dataloader.dataset.datasets[0]
    if cfg.train_dataloader.dataset['type'] == 'CBGSDataset':
        cfg.train_dataloader.dataset = cfg.train_dataloader.dataset.dataset

    train_data_cfg = cfg.train_dataloader.dataset

    if aug:
        show_pipeline = cfg.train_pipeline
    else:
        show_pipeline = cfg.test_pipeline
        for i in range(len(cfg.train_pipeline)):
            if cfg.train_pipeline[i]['type'] == 'LoadAnnotations3D':
                show_pipeline.insert(i, cfg.train_pipeline[i])
            # Collect data as well as labels
            if cfg.train_pipeline[i]['type'] == 'Pack3DDetInputs':
                if show_pipeline[-1]['type'] == 'Pack3DDetInputs':
                    show_pipeline[-1] = cfg.train_pipeline[i]
                else:
                    show_pipeline.append(cfg.train_pipeline[i])

    train_data_cfg['pipeline'] = show_pipeline

    return cfg

In [5]:
from sensus import configs

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[Open3D INFO] Resetting default logger to print to terminal.


In [ ]:
190

In [6]:
config_path = osp.join(configs.__path__[0], '_base_', 'datasets',
    'dair-infrastructure-3d-3class.py')

In [7]:
cfg = build_data_cfg(config_path, False, None)
init_default_scope(cfg.get('default_scope', 'mmdet3d'))
vis_task = 'lidar_det'

In [8]:
dataset = DATASETS.build(
    cfg.train_dataloader.dataset,
    default_args=dict(filter_empty_gt=False))

02/24 23:56:56 - mmengine - INFO - ------------------------------
02/24 23:56:56 - mmengine - INFO - The length of training dataset: 8800
02/24 23:56:56 - mmengine - INFO - The number of instances per category in the dataset:
+------------+--------+
| category   | number |
+------------+--------+
| Pedestrian | 19444  |
| Cyclist    | 9304   |
| Car        | 94936  |
+------------+--------+


In [9]:
for i, item in enumerate(dataset):
    # the 3D Boxes in input could be in any of three coordinates
    data_input = item['inputs']
    data_sample = item['data_samples'].numpy()
    if i == 190:
        break

In [16]:
data_sample.keys()

['gt_instances_3d', 'gt_pts_seg', 'gt_instances', 'eval_ann_info']

In [26]:
data_sample.gt_instances_3d.bboxes_3d.tensor

tensor([[ 88.3596,  -9.4646,  -1.8232,   1.8111,   4.2418,   2.3550,  -1.5654],
        [ 65.8941,   9.5356,  -1.6752,   1.9200,   4.2792,   1.6270,   1.5677],
        [ 43.6112, -21.2305,  -1.6285,   0.6517,   1.7719,   1.6144,  -1.5503],
        [ 44.3531, -28.1597,  -1.5563,   1.8030,   4.2799,   1.4030,  -3.1358],
        [ 33.8514, -24.2825,  -1.6833,   1.9026,   4.5430,   1.5883,  -0.5310],
        [ 63.2716, -15.6731,  -1.8126,   0.7301,   1.8201,   1.5227,   1.3153],
        [ 24.6383, -18.3547,  -1.7647,   1.8457,   4.2715,   1.6311,  -1.2149]])

In [27]:
a = data_sample.gt_instances_3d.bboxes_3d.tensor

In [39]:
def flip_horiz_roty(bboxes):
    x = np.pi - bboxes[:, 6]
    x = np.arctan2(np.sin(x), np.cos(x))
    bboxes[:, 6] = x
    return bboxes

In [40]:
# Print torch tensor with decimal places instead of scientific notation
np.set_printoptions(precision=3, suppress=True)
flip_horiz_roty(a)

tensor([[ 8.8360e+01, -9.4646e+00, -1.8232e+00,  1.8111e+00,  4.2418e+00,
          2.3550e+00, -1.5762e+00],
        [ 6.5894e+01,  9.5356e+00, -1.6752e+00,  1.9200e+00,  4.2792e+00,
          1.6270e+00,  1.5738e+00],
        [ 4.3611e+01, -2.1230e+01, -1.6285e+00,  6.5175e-01,  1.7719e+00,
          1.6144e+00, -1.5913e+00],
        [ 4.4353e+01, -2.8160e+01, -1.5563e+00,  1.8030e+00,  4.2799e+00,
          1.4030e+00, -5.7562e-03],
        [ 3.3851e+01, -2.4283e+01, -1.6833e+00,  1.9026e+00,  4.5430e+00,
          1.5883e+00, -2.6106e+00],
        [ 6.3272e+01, -1.5673e+01, -1.8126e+00,  7.3014e-01,  1.8201e+00,
          1.5227e+00,  1.8263e+00],
        [ 2.4638e+01, -1.8355e+01, -1.7647e+00,  1.8457e+00,  4.2715e+00,
          1.6311e+00, -1.9267e+00]])

In [37]:
def wrap_to_pi(x):
    return np.arctan2(np.sin(x), np.cos(x))

In [38]:
x = -1.2
wrap_to_pi(-(x - np.pi/2) + np.pi/2)

-1.9415926535897932